## Take a bunch of runs and re-do them with 

In [1]:
from matplotlib import rc
import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import os
import xarray as xr
import subprocess
import matplotlib.pyplot as plt
import shutil
from pathlib import Path
os.chdir("/home/149/ab8992/topographic-NIWs")
basepath = Path.cwd().absolute()

import autolib as al

# from dask.distributed import Client
# client = Client()
# client

## Make common forcing

In [2]:
import autolib as al


### COMMON ACROSS ALL EXPERIMENTS. 
##########################################################
nlayers = 20             # number of layers
Lx = 4000                # domain zonal extent [km]
Ly = 4000                # domain meridional extent [km]
H  = 4000                # total fluid's depth in [m]
gridspacing = 2          # in [km]
flat = False
reverse = True
x = np.arange(-Lx/2, Lx/2, gridspacing)
y = np.arange(-Ly/2, Ly/2, gridspacing)
ninterf = nlayers + 1    # number of interfaces
interfaces = np.arange(0,  ninterf)
nx = int(round(Lx/gridspacing))
ny = int(round(Ly/gridspacing))
overrides = ["ADIABATIC = True",
             "RHO_0 = 1027.0",
             f"NJGLOBAL={ny}",
             f"NIGLOBAL={nx}",
             f"LENLON = {Lx}.0",
             f"WESTLON = -{Lx//2}.0",
             f"LENLAT = {Ly}.0",
             f"WESTLAT = -{Ly//2}.0",
             f"NK = {nlayers + 1}"] ## Add one for mixed layer
default_dir = "default_rundir"


## Default values when unperturbed
default_height = 1000 # m
default_forcing_latwidth = 100 # km
default_duration = 10 # hours
default_layerdensities = np.linspace(1027,1029,nlayers) ## This is the default stratification. Has be modified to add mixed layer!
default_strength = 1
default_ridge_width = 12.5 # km


## Value s from first pass! 
## Perterbation values
## strat variables 
# strats = [0.25,0.2,0.5,0.75,1,1.25,1.5,2,3,4,0.001,0.01,6]

## Height
heights = [10,20,50,80,120,150,225,275,350,450,500,600,650,700,750,1250,1500,2000]
# heights = [1000]
## Duration
durations = np.linspace(1,10,19) ## Note! Will need to modify the amount of time that model runs for with forcing in this case

## Strength
strengths = np.arange(1,15)

## Topog Width
widths = [4,8,16,25,35,50,75,100,125,150,175,200]

## Forcing Width
forcing_widths = [10,18,25,50,75,150,175,200,225,250,275,300,350,400,450,500]



# # Save the common wind forcing
# STRESS_X = al.windstress_gaussian(nx = nx,ny = ny)
# eta_ridge = al.eta_gaussian_hill(
#         nlayers=nlayers,
#         ridge=True,
#         nx = nx,
#         ny = ny,
#         )
   
# al.save_inputdata(
#     x,
#     y,
#     STRESS_X,
#     STRESS_X * 0,
#     eta_ridge,
#     "common",
#     save_wind = False
#     )


In [3]:
expt = al.expt(x,y,nlayers,"common","common",overrides=overrides)
expt.setup()
expt.run()

RUNNING: common_common
laboratory path:  /scratch/v45/ab8992/mom6
binary path:  /scratch/v45/ab8992/mom6/bin
input path:  /scratch/v45/ab8992/mom6/input
work path:  /scratch/v45/ab8992/mom6/work
archive path:  /scratch/v45/ab8992/mom6/archive
payu: work path already exists.
      Sweeping as --force option is True.
Moving log common_common.o125172139
Moving log common_common.e125172139
Removing work path /scratch/v45/ab8992/mom6/work/common_common
Removing symlink /home/149/ab8992/topographic-NIWs/rundirs/common/common_common/work
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
Setting up mom6
Checking exe and input manifests
Updating full hashes for 1 files in manifests/exe.yaml
Creating restart manifest
Writing manifests/restart.yaml
Writing manifests/exe.yaml
125172311.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input ma

In [6]:
import subprocess
# heights = [10]
for height in heights:
    expt = al.expt(x,y,nlayers,"height",height,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    # expt.run()
    expt.resume()

Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


...

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 6
125223532.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P g40 -l walltime=0:30:00 -l ncpus=912 -l mem=3648GB -N height_1250 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_N_RUNS=4,PAYU_FORCE=True,MODULESHOME=/opt/Modules/v4.3.0,MODULES_CMD=/opt/Modules/v4.3.0/libexec/modulecmd.tcl,MODULEPATH=/etc/scl/modulefiles:/opt/Modules/modulefiles:/opt/Modules/v4.3.0/modulefiles:/apps/Modules/modulefiles -W umask=027 -l storage=gdata/hh5+gdata/v45+scratch/v45 -- /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/bin/python3.10 /g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu-run
Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


In [14]:
for duration in durations:
    expt = al.expt(x,y,nlayers,"duration",duration,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    # al.overwrite_in_file(
    #     str(path / "input.nml"),
    #     "    hours = ",
    #     f"    hours = {int(2 * duration)}"

    # )
    # expt.run()
    expt.resume()
    




Total outputs: 11


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 11


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 11


...

Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 9
125224117.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P g40 -l walltime=3600 -l ncpus=912 -l mem=3648GB -N duration_7.5 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_N_RUNS=1,PAYU_FORCE=True,MODULESHOME=/opt/Modules/v4.3.0,MODULES_CMD=/opt/Modules/v4.3.0/libexec/modulecmd.tcl,MODULEPATH=/etc/scl/modulefiles:/opt/Modules/modulefiles:/opt/Modules/v4.3.0/modulefiles:/apps/Modules/modulefiles -W umask=027 -l storage=gdata/hh5+gdata/v45+scratch/v45 -- /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/bin/python3.10 /g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu-run
Total outputs: 9
125224118.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: war

In [7]:
for strength in strengths:
    expt = al.expt(x,y,nlayers,"strength",strength,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # path = al.basepath / "rundirs" / expt.variable / f"{expt.variable}_{expt.var_value}"
    # expt.run()
    expt.resume()

Total outputs: 0
125223537.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P ol01 -l walltime=01:00:00 -l ncpus=912 -l mem=3648GB -N strength_0 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin,PAYU_N_RUNS=10,PAYU_FORCE=True,MODULESHOME=/opt/Modules/v4.3.0,MODULES_CMD=/opt/Modules/v4.3.0/libexec/modulecmd.tcl,MODULEPATH=/etc/scl/modulefiles:/opt/Modules/modulefiles:/opt/Modules/v4.3.0/modulefiles:/apps/Modules/modulefiles -W umask=027 -l storage=gdata/hh5+gdata/v45+scratch/v45 -- /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/bin/python3.10 /g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu-run
Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


...

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


In [9]:
for width in widths:
    expt = al.expt(x,y,nlayers,"width",width,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # expt.run()
    expt.resume()

Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


...

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


Total outputs: 10


Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3/bin/payu", line 10, in <module>
    sys.exit(parse())
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 42, in parse
    run_cmd(**args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/subcommands/run_cmd.py", line 24, in runcmd
    pbs_vars = cli.set_env_vars(init_run=init_run,
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/payu/cli.py", line 122, in set_env_vars
    assert n_runs > 0
AssertionError


In [3]:
# forcing_widths = [250,275,300,350,400,450,500]

for forcing_width in forcing_widths:
    expt = al.expt(x,y,nlayers,"forcing_width",forcing_width,overrides=overrides)
    # expt.make_inputs()
    # expt.setup()
    # expt.run()
    expt.resume()

Total outputs: 6
125364050.gadi-pbs
payu: warning: Job request includes 12 unused CPUs.
payu: warning: CPU request increased from 900 to 912
Loading input manifest: manifests/input.yaml
Loading restart manifest: manifests/restart.yaml
Loading exe manifest: manifests/exe.yaml
payu: Found modules in /opt/Modules/v4.3.0
qsub -q normal -P ol01 -l walltime=3600 -l ncpus=912 -l mem=3648GB -N forcing_width_1 -l wd -j n -v PAYU_PATH=/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/bin,PAYU_N_RUNS=4,PAYU_FORCE=True,MODULESHOME=/opt/Modules/v4.3.0,MODULES_CMD=/opt/Modules/v4.3.0/libexec/modulecmd.tcl,MODULEPATH=/etc/scl/modulefiles:/opt/Modules/modulefiles:/opt/Modules/v4.3.0/modulefiles:/apps/Modules/modulefiles -W umask=027 -l storage=gdata/hh5+gdata/v45+scratch/v45 -- /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/bin/python3.10 /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.01/bin/payu-run
Total outputs: 6
125364051.gadi-pbs
payu: warning: Job request includes 12 unused

## Check for runs that are missing postprocessing

In [20]:
from pathlib import Path

def save_data(expt,run = "*"):

    basepath = Path("/home/149/ab8992/topographic-NIWs/rundirs") / expt.split("_")[0] / expt / "archive" / f"output{run}"
    print("basepath ",basepath)
    u = xr.open_mfdataset(str(basepath) + "/u.nc", decode_times = False,decode_cf = False).sel(yh = slice(-250,250)).sel(xq = 250,method = "nearest").u.load()
    ufar = xr.open_mfdataset(str(basepath) + "/u.nc", decode_times = False,decode_cf = False).sel(yh = slice(-250,250)).isel(xq = 0).u.load()
    v = xr.open_mfdataset(str(basepath) + "/v.nc", decode_times = False,decode_cf = False).sel(xh = slice(-250,250)).sel(yq = 250,method = "nearest").v.load()

    eEastFar = xr.open_mfdataset(str(basepath) + "/e.nc", decode_times = False,decode_cf = False).sel(yh = slice(-250,250)).isel(xh = 0).e.load()

    eEast = xr.open_mfdataset(str(basepath) + "/e.nc", decode_times = False,decode_cf = False).sel(yh = slice(-250,250)).sel(xh = 250,method = "nearest").e.load()
    eNorth = xr.open_mfdataset(str(basepath) + "/e.nc", decode_times = False,decode_cf = False).sel(xh = slice(-250,250)).sel(yh = 250,method = "nearest").e.load()

    pEast = al.calculate_pressure(eEast,u.zl)
    pEastFar = al.calculate_pressure(eEastFar,u.zl)
    pNorth = al.calculate_pressure(eNorth,u.zl)

    pEast = pEast - pEast.isel(time = 0)
    pEastFar = pEastFar - pEastFar.isel(time = 0)
    pNorth = pNorth - pNorth.isel(time = 0)

    out = xr.merge([u - ufar,v,(pEast - pEastFar).rename("pEast"),pNorth.rename("pNorth")]).load()
    outpath = Path("/g/data/v45/ab8992/bottom-niws-outputs/sept2024-bniw-outputs/") / expt.split("_")[0] / expt
    if not os.path.exists(outpath):
        os.makedirs(outpath)
    if run == "*":
        run = "all"
    out.to_netcdf(outpath / f"{expt}_{run}.nc")
    return 

def re_run_postprocessing(exptname,runs):
    archive = Path("/home/149/ab8992/topographic-NIWs/rundirs") / exptname 
    outpath = Path("/g/data/v45/ab8992/bottom-niws-outputs/sept2024-bniw-outputs") / exptname
    for run in runs:
        outputs = list((archive / f"{exptname}_{run}" / "archive").glob("output*"))
        for output in outputs:
            num = int(output.name.split("output")[-1])
            # print(outpath / f"{exptname}_{run}" / f"{exptname}_{run}_{num:03}.nc")
            if not os.path.exists(outpath / f"{exptname}_{run}" / f"{exptname}_{run}_{num:03}.nc"):
                print(num,output.name,run)
                save_data(f"{exptname}_{run}",run = f"{num:03}")


re_run_postprocessing("width",widths)
# re_run_postprocessing("duration",strengths)
    # save_data(rundir.name,run = f"{(len(outputs) - 1):03}" )

# /g/data/v45/ab8992/bottom-niws-outputs/sept2024-bniw-outputs/strength/strength_9/:
# strength_9_000.nc  strength_9_001.nc  strength_9_002.nc